In [7]:
from dataset import generate
from cluster import cluster_ip
from lxml import etree
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd
import math
import numpy as np
import fnmatch
#nltk.install("all")

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

import nlp

from segmentador import Segmentador
from tp2_modulos import CleanText

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

In [16]:


actual_directory = os.getcwd()

ruta = os.path.join(actual_directory, "mini_corpus")
ruta_del_directorio = str(ruta) + "\\*.txt"
# Me produce una lista con los nombres de los archivos txt del corpus
textos_originales = glob.glob(ruta_del_directorio)


if not os.path.exists(str(actual_directory)+"\\cleaned_corpus"):
    os.mkdir(str(actual_directory)+'\\cleaned_corpus')
    
cleaned_corpus = str(actual_directory)+"\\cleaned_corpus"

# De esta manera le voy robo el nombre a los xml
path_xml = str(actual_directory) + "\\mini_corpus" + "\\*.xml"
xml_docs = glob.glob(path_xml)


if not os.path.exists(str(actual_directory)+"\\sospechoso"):
    os.mkdir(str(actual_directory)+'\\sospechoso')
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(str(actual_directory)+"\\sospechoso")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory)+"\\sospechoso", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

    
if not os.path.exists(str(actual_directory) + "\\xml\\output_xml"):
    os.mkdir(str(actual_directory) + "\\xml\\output_xml")
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(str(actual_directory) + "\\xml\\output_xml")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory) + "\\xml\\output_xml", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")


sospechosos_path = str(actual_directory) + "\\sospechoso\\"
output_xml = str(actual_directory) + "\\xml\\output_xml\\"


Se elimino carpeta vieja de fragmentos sospechosos
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00001.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00002.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00003.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00004.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00005.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00006.txt
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\sospechoso\sospsuspicious-document00007.txt
Se elimino carpeta vieja de fragmentos sospechosos en formato xml
Eliminado: c:\Users\pbonafe\Diplo\mentoria\M11-DeteccionPlagio-GrupoB\xml\output_xml\

In [17]:




# Comienzo a leer el conjunto de textos
for i, archive in enumerate(textos_originales):
    
    print("File: " + str(i+1))
    
    f = open(archive, encoding="utf-8-sig")
    # Leo contenido del archivo
    text = f.read()

    # Cierro el txt
    f.close()           
    
    # Nombre de Archivo
    filename = os.path.basename(archive)
    # Separo nombre de archivo y extension
    filename2, extension = os.path.splitext(filename)
    
    Seg = Segmentador(text)
    # La variable data_text posee el texto completo, separado en párrafos
    data_text = Seg.content_segmentation()
    


    data_list = []

    lista_to_df = []
    
    for j, segmento in enumerate(data_text):

        segmento_limpio = CleanText(str(segmento)).lemmatizeText()

        data_list.append([j, re.sub("\n", " ", segmento_limpio), len(re.sub("\n", " ", segmento_limpio))])

        lista_to_df.append(
            [j, filename, nlp.getnumOfPunctN(segmento_limpio), nlp.gettypeToken(segmento_limpio)]
        )                        
    
    df = pd.DataFrame(lista_to_df, columns=["index", "filename", "getnumOfPunctN", "gettypeToken"])
    
    
    #df["getfleshReadingEase"] = df["getfleshReadingEase"] / df["getfleshReadingEase"].max()
    
    # Seleccionar los valores numéricos del df
    # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
    dataframe = df.iloc[:,[2,3]]
    
    # Aprendizaje No Supervisado
    cluster = cluster_ip(dataframe, 4, "cosine")
    labels = cluster.labels_    
    df["labels"] = labels
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum=len(set(labels))
    n_noise = list(labels).count(-1) 
    
    print("Estimated number of clusters: %d" % realClusterNum)
    print("Estimated number of noise points: %d" % n_noise)
    counts = df["labels"].value_counts().to_dict()
    print(counts)
    
    # Me sirve para guardar los casos de plagio y compararlos
    xml_name = os.path.basename(xml_docs[i])
    xml_name2, extension = os.path.splitext(xml_name)
    
    # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
    sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")
    
    # Los fragmentos que fueron clasificados como plagio
    # Se guardan en el archivo .txt
    # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
    # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
    for x in range(len(df["labels"])):
        l = df["labels"][x]
        if l == -1:
            print(data_list[x][1])
            print(x)
            sherlock.write(data_list[x][1]+os.linesep)       
               
    sherlock.close()    
    
    document_el = etree.Element("document", reference = xml_name2 + ".txt")

    offset = []

    init = 0

    sum_length = []

    last = -1

    for y in range(len(df["labels"])):       

        l = df["labels"][y]

        if l == -1:

            print(y)

            if y == 0:

                sum_length.append([init, data_list[y][2]])

                print(f"Case 0, y = 0")

            elif y != 0 and (y == (last + 1) or len(sum_length) == 0):

                sum_length.append([init, data_list[y][2]])

                print(f"Case 1, y == (last + 1)")

            elif y != 0 and y != (last + 1):

                this_length = sum([l[1] for l in sum_length])

                feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))

                sum_length.clear()

                print(sum_length)

                sum_length.append([init, data_list[y][2]])

                print(f"Case 2, y != (last + 1)")   

            last = y

        if y == (len(df["labels"]) - 1) and len(sum_length) > 0:

            print("Entro2")

            this_length = sum([l[1] for l in sum_length])

            feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(sum_length[0][0]), this_length=str(this_length))


        init += data_list[y][2] + 1





    et = etree.ElementTree(document_el)
    with open(output_xml+xml_name, "wb") as fe:
        et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)

File: 1
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 38}
File: 2
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 271}
File: 3
Estimated number of clusters: 1
Estimated number of noise points: 1
{0: 58, -1: 1}
  
58
58
Case 1, y == (last + 1)
Entro2
File: 4
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 22}
File: 5
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 30}
File: 6
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 228}
File: 7
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 7}
File: 8
Estimated number of clusters: 1
Estimated number of noise points: 1
{0: 869, -1: 1}
  
869
869
Case 1, y == (last + 1)
Entro2
File: 9
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 8}
File: 10
Estimated number of clusters: 1
Estimated number of noise points: 0
{0: 586}
